# 06-02 - Process - Plugin

## Environment

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.38.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.38.0-alpha"
#r "nuget: DotNetEnv, 3.1.1"

using System.ClientModel; 
using System.ComponentModel;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using DotNetEnv;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

public class Configuration {
    public string ApiKey { get; set; } = Env.GetString("SK_OPENAI_APIKEY");
    public string Endpoint { get; set; } = Env.GetString("SK_OPENAI_ENDPOINT");
    public string ChatCompletionDeploymentDefault { get; set; } = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");
}

Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0 Microsoft.SemanticKernel.Process.Core, 1.38.0-alpha Microsoft.SemanticKernel.Process.LocalRuntime, 1.38.0-alpha

Configuration loaded...


## Define Process Step Functionality


In [2]:
#pragma warning disable SKEXP0080

public class RetrieveWinnerProcessStep : KernelProcessStep
{
    public class RetrieveWinner : KernelProcessStep
    {
        [KernelFunction("get_sport_event_winner")]
        [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
        public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
        {
            
            Console.WriteLine($"\tPlugin: 'GetSportEventWinner({sportEventName}, {sportEventYear})' called...");
            Console.WriteLine($"\tPlugin: 'GetSportEventWinner({sportEventName}, {sportEventYear})' returning 'Munich Flying Dolphins'...");
            
            // Implement the logic to get the winner of the sport event.
            return "Munich Flying Dolphins";
        }
    }


    [KernelFunction]
    public async Task<string> QueryForWinner(Kernel kernel, string sportEventName)
    {
        Console.WriteLine($"\n'QueryForWinner({sportEventName}) called...");
        
        IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

        string prompt = @$"
            Who won the {sportEventName}? 
            Just return the name of the winning team.
        ";
        ChatHistory chatHistory = new ChatHistory();
        chatHistory.AddUserMessage(prompt);
        
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");

        KernelFunction getSportEventWinner = kernel.Plugins.GetFunction("RetrieveWinner", "get_sport_event_winner");
        PromptExecutionSettings promptExecutionSettings = new PromptExecutionSettings() {
            FunctionChoiceBehavior = FunctionChoiceBehavior.Required(
                new List<KernelFunction>() { getSportEventWinner }  
            )
        };
        promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();
        
        Console.WriteLine($"\t'ChatCompletionService' called...");
        ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
            chatHistory: chatHistory, 
            executionSettings: promptExecutionSettings,
            kernel: kernel
        );

        Console.WriteLine($"\t'ChatCompletionService' returned {chatMessageContent.Content}...");
        return $"Winner: {chatMessageContent.Content ?? ""} - Event: {sportEventName}";
    }
 
}

Console.WriteLine($"'RetrieveWinnerProcessStep' defined...");

'RetrieveWinnerProcessStep' defined...


In [3]:
#pragma warning disable SKEXP0080

public class RetrieveScoreProcessStep : KernelProcessStep
{

    public class RetrieveScore
    {
        [KernelFunction("get_sport_event_score")]
        //Description is just necessary if function name is cryptic and not self-explanatory
        [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
        public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
        {
            Console.WriteLine($"\tPlugin: 'GetSportEventScore({sportEventName}, {sportEventYear})' called...");
            Console.WriteLine($"\tPlugin: 'GetSportEventScore({sportEventName}, {sportEventYear})' returning '24:1'...");
            // Implement the logic to get the result of the sport event.
            return "24:1";
        }
    }

    [KernelFunction]
    public async Task<string> QueryForScore(Kernel kernel, string sportEventName)
    {

        Console.WriteLine($"\n'QueryForScore({sportEventName}) called...");
        
        IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

        string prompt = @$"
            	What was the score of the {sportEventName}? 
                Just return the score."
        ;
        ChatHistory chatHistory = new ChatHistory();
        chatHistory.AddUserMessage(prompt);
        
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveScore>("RetrieveScore");

        KernelFunction getSportEventScore = kernel.Plugins.GetFunction("RetrieveScore", "get_sport_event_score");
        PromptExecutionSettings promptExecutionSettings = new PromptExecutionSettings() {
            FunctionChoiceBehavior = FunctionChoiceBehavior.Required(
                new List<KernelFunction>() { getSportEventScore }  
            )
        };
        promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Required();
        
        Console.WriteLine($"\t'ChatCompletionService' called...");       
        ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
            chatHistory: chatHistory, 
            executionSettings: promptExecutionSettings,
            kernel: kernel
        );
        Console.WriteLine($"\t'ChatCompletionService' returned: {chatMessageContent.Content}...");
        
        return chatMessageContent.Content ?? "";
    }

}  

Console.WriteLine($"'RetrieveScoreProcessStep' definied...");


'RetrieveScoreProcessStep' definied...


In [4]:
#pragma warning disable SKEXP0080

public class StoreResultHelper : KernelProcessStep
{
    static List<string> _storage = new List<string>();

    [KernelFunction]
    public async Task StoreResult(string result)
    {
        // Implement the logic to store the result.
        await Task.Run( () => _storage.Add(result));
    }

    [KernelFunction]
    public async Task DisplayResults(string result)
    {
        Console.WriteLine($"\nProcess results:");
        await Task.Run( () => {
            foreach (string result in _storage)
            {
                Console.WriteLine($"\tStored result: {result}");
            }
            Console.WriteLine($"\tStored result: {result}");
            _storage.Clear();
        });
    }
}

Console.WriteLine($"'StoreResultHelper' defined...");

'StoreResultHelper' defined...


## Process

In [5]:
#pragma warning disable SKEXP0080

// Create the process builder
ProcessBuilder processBuilder = new("RetrieveSportEventResult");

// Add the steps
ProcessStepBuilder retrieveWinnerProcessStep = processBuilder.AddStepFromType<RetrieveWinnerProcessStep>();
ProcessStepBuilder retrieveScoreProcessStep = processBuilder.AddStepFromType<RetrieveScoreProcessStep>();
ProcessStepBuilder storeResultProcessStep = processBuilder.AddStepFromType<StoreResultHelper>();


processBuilder
    .OnInputEvent("RetrieveSportEventResult")
    .SendEventTo(new (retrieveWinnerProcessStep));

retrieveWinnerProcessStep
    .OnFunctionResult()
    .SendEventTo(new (retrieveScoreProcessStep))
    .SendEventTo(new (storeResultProcessStep, functionName: "StoreResult"));

retrieveScoreProcessStep
    .OnFunctionResult()
    .SendEventTo(new (storeResultProcessStep, functionName: "DisplayResults"));


KernelProcess kernelProcess = processBuilder.Build();

Console.WriteLine($"Process defined ...");

Process defined ...


## Kernel / Process Start

In [6]:
#pragma warning disable SKEXP0080

Configuration _configuration = new Configuration();
IKernelBuilder kernelBuilder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        apiKey: _configuration.ApiKey, 
        endpoint: _configuration.Endpoint, 
        deploymentName: _configuration.ChatCompletionDeploymentDefault
    );
kernelBuilder.Services.AddSingleton<Configuration>(_configuration);
Kernel kernel = kernelBuilder.Build();

KernelProcessContext kernelProcessContext = await kernelProcess.StartAsync(
    kernel, 
    new KernelProcessEvent { 
        Id = "RetrieveSportEventResult", 
        Data = "Super Sports Championship 2025" 
    }
);

Console.WriteLine($"\nProcess finished...");


'QueryForWinner(Super Sports Championship 2025) called...
	'ChatCompletionService' called...
	Plugin: 'GetSportEventWinner(Super Sports Championship, 2025)' called...
	Plugin: 'GetSportEventWinner(Super Sports Championship, 2025)' returning 'Munich Flying Dolphins'...
	'ChatCompletionService' returned Munich Flying Dolphins...

'QueryForScore(Winner: Munich Flying Dolphins - Event: Super Sports Championship 2025) called...
	'ChatCompletionService' called...
	Plugin: 'GetSportEventScore(Super Sports Championship, 2025)' called...
	Plugin: 'GetSportEventScore(Super Sports Championship, 2025)' returning '24:1'...
	'ChatCompletionService' returned: 24:1...

Process results:
	Stored result: Winner: Munich Flying Dolphins - Event: Super Sports Championship 2025
	Stored result: 24:1

Process finished...
